In [1]:
import pandas as pd
import unicodedata
import requests

# #send to rest API
import json 
import requests
from requests.auth import HTTPBasicAuth

In [2]:
#batter_bases

In [3]:
under_runs_rate=pd.read_csv('undernewruns.csv')
over_runs_rate=pd.read_csv('overnewruns.csv')
def_rank = pd.read_csv('../scrapers/def_scraper/batter_runs_def_rank.csv')
batter_team_2025 = pd.read_csv('../general/stats/batter_team_2025.csv')

In [4]:
batter_team_2025.rename(columns={'Team': 'team', 'Player':'player'}, inplace=True)


In [5]:
team_mapping_5 = {
    'CHC': 'CHC',
    'LAD': 'LAD',
    'ARI': 'ARI',
    'PIT': 'PIT',
    'MIA': 'MIA',
    'TOR': 'TOR',
    'BOS': 'BOS',
    'ATH': 'ATH',
    'BAL': 'BAL',
    'SEA': 'SEA',
    'SDP': 'SDP',
    'ATL': 'ATL',
    'PHI': 'PHI',
    'TEX': 'TEX',
    'MIL': 'MIL',
    'WSN': 'WSH',
    'NYY': 'NYY',
    'STL': 'STL',
    'TBR': 'TBR',
    'DET': 'DET',
    'KCR': 'KCR',
    'CLE': 'CLE',
    'COL': 'COL',
    'NYM': 'NYM',
    'CIN': 'CIN',
    'MIN': 'MIN',
    'SFG': 'SFG',
    'HOU': 'HOU',
    'CHW': 'CHW',
    'LAAs': 'LAA'
}

batter_team_2025['team'] = batter_team_2025['team'].map(team_mapping_5)

In [6]:
def_rank.rename(columns={'Team': 'team'}, inplace=True)

In [7]:
def_rank.rename(columns={'Rank': 'def_rank'}, inplace=True)

In [8]:
team_mapping = {
    'Milwaukee Brewers': 'MIL',
    'Minnesota Twins': 'MIN',
    'Chicago Cubs': 'CHC',
    'Colorado Rockies': 'COL',
    'Washington Nationals': 'WSH',
    'Toronto Blue Jays': 'TOR',
    'Atlanta Braves': 'ATL',
    'Baltimore Orioles': 'BAL',
    'New York Yankees': 'NYY',
    'Philadelphia Phillies': 'PHI',
    'Pittsburgh Pirates': 'PIT',
    'Kansas City Royals': 'KCR',
    'Arizona Diamondbacks': 'ARI',
    'Boston Red Sox': 'BOS',
    'Detroit Tigers': 'DET',
    'Cleveland Guardians': 'CLE',
    'Tampa Bay Rays': 'TBR',
    'San Francisco Giants': 'SFG',
    'Los Angeles Dodgers': 'LAD',
    'Seattle Mariners': 'SEA',
    'Cincinnati Reds': 'CIN',
    'Texas Rangers': 'TEX',
    'Chicago White Sox': 'CHW',
    'Los Angeles Angels': 'LAA',
    'Miami Marlins': 'MIA',
    'New York Mets': 'NYM',
    'Athletics': 'ATH',
    'St. Louis Cardinals': 'STL',
    'San Diego Padres': 'SDP',
    'Houston Astros': 'HOU'
}

def_rank['team'] = def_rank['team'].map(team_mapping)

In [9]:
#fixing the accentes on the names
def normalize_name(name):
    if not isinstance(name, str):
        return name
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')

In [10]:
batter_team_2025['player'] = batter_team_2025['player'].apply(normalize_name)

In [11]:
#cleaning the # and *
def clean_name_symbols(name):
    if not isinstance(name, str):
        return name
    return name.replace('*', '').replace('#', '')

batter_team_2025['player'] = batter_team_2025['player'].apply(clean_name_symbols)

In [12]:
# multi_team_flags = ['2TM', '3TM', '4TM', '5TM']
# filtered_df_2 = batter_team_2025[~batter_team_2025['team'].isin(multi_team_flags)].copy()


# # Step 2: Keep only the LAST team listed for each player (i.e., current team)
# batter_team_2025 = filtered_df_2.drop_duplicates(subset='player', keep='last').reset_index(drop=True)

In [13]:
headers = {
    'X-Api-Key': 'fac7e1a69f2baa47da395bde36dc869c'  
    }

url = 'https://api.sportsgameodds.com/v2/events/'

next_cursor = None
event_data = []

while True:
    try:
        response = requests.get(
            url,
            headers=headers,
            params={
                "sportID":"BASEBALL",
                "leagueID": "MLB",
                "oddIDs": "points-PLAYER_ID-game-ou-over",
                "includeOpposingOddIDs": "true", 
                "oddsAvailable":"true",
                "cursor": next_cursor,
                "include":"teams",
                'started':"false",

            },
        )

        response.raise_for_status()  

        data = response.json()
        event_data.extend(data.get("data", []))
        next_cursor = data.get("nextCursor")

        if not next_cursor:
            break

    except requests.exceptions.RequestException as e:
        print("Error fetching events:", e)
        break



In [14]:
event_data

[{'eventID': '31Qcqt0rYjSZKKkvvSYK',
  'sportID': 'BASEBALL',
  'leagueID': 'MLB',
  'type': 'match',
  'teams': {'home': {'teamID': 'PITTSBURGH_PIRATES_MLB',
    'names': {'long': 'Pittsburgh Pirates',
     'short': 'PIT',
     'medium': 'Pirates'},
    'colors': {'primary': '#000000', 'primaryContrast': '#FFFFFF'},
    'statEntityID': 'home'},
   'away': {'teamID': 'CINCINNATI_REDS_MLB',
    'names': {'long': 'Cincinnati Reds', 'short': 'CIN', 'medium': 'Reds'},
    'colors': {'primary': '#C80021', 'primaryContrast': '#FFFFFF'},
    'statEntityID': 'away'}},
  'odds': {'points-SPENCER_HORWITZ_1_MLB-game-ou-over': {'oddID': 'points-SPENCER_HORWITZ_1_MLB-game-ou-over',
    'opposingOddID': 'points-SPENCER_HORWITZ_1_MLB-game-ou-under',
    'marketName': 'Spencer Horwitz Runs Over/Under',
    'statID': 'points',
    'statEntityID': 'SPENCER_HORWITZ_1_MLB',
    'periodID': 'game',
    'betTypeID': 'ou',
    'sideID': 'over',
    'playerID': 'SPENCER_HORWITZ_1_MLB',
    'started': False,
 

In [15]:
odds_list = []

for event in event_data:
    event_id = event.get("eventID", "Unknown Event")
    teams = sorted(set(player_data.get("teamID") for player_data in event.get("players", {}).values() if player_data.get("teamID")))
    
    odds = event.get("odds", {})
    
    for odd_id, odd_object in odds.items():
        base_row = {
            "eventID": event_id,
            "teams": ", ".join(teams),
            "oddID": odd_object.get("oddID"),
            "opposingOddID": odd_object.get("opposingOddID"),
            "marketName": odd_object.get("marketName"),
            "playerID": odd_object.get("statEntityID"),
            "side": odd_object.get("sideID"),
            "line": odd_object.get("bookOverUnder", odd_object.get("fairOverUnder", "N/A")),
            "score": odd_object.get("score", "N/A"),
        }
        
        by_bookmaker = odd_object.get("byBookmaker", {})
        
        for bookmaker, book_data in by_bookmaker.items():
            row = base_row.copy()
            row["bookmaker"] = bookmaker
            row["odds"] = book_data.get("odds", "N/A")
            row["overUnder"] = book_data.get("overUnder", "N/A")
            odds_list.append(row)

df_odds = pd.DataFrame(odds_list)


In [16]:
df_odds

,eventID,teams,oddID,opposingOddID,marketName,playerID,side,line,score,bookmaker,odds,overUnder
0,31Qcqt0rYjSZKKkvvSYK,"CINCINNATI_REDS_MLB, PITTSBURGH_PIRATES_MLB",points-SPENCER_HORWITZ_1_MLB-game-ou-over,points-SPENCER_HORWITZ_1_MLB-game-ou-under,Spencer Horwitz Runs Over/Under,SPENCER_HORWITZ_1_MLB,over,0.5,N/A,draftkings,+195,0.5
1,31Qcqt0rYjSZKKkvvSYK,"CINCINNATI_REDS_MLB, PITTSBURGH_PIRATES_MLB",points-SPENCER_HORWITZ_1_MLB-game-ou-over,points-SPENCER_HORWITZ_1_MLB-game-ou-under,Spencer Horwitz Runs Over/Under,SPENCER_HORWITZ_1_MLB,over,0.5,N/A,fliff,+190,0.5
2,31Qcqt0rYjSZKKkvvSYK,"CINCINNATI_REDS_MLB, PITTSBURGH_PIRATES_MLB",points-SPENCER_HORWITZ_1_MLB-game-ou-under,points-SPENCER_HORWITZ_1_MLB-game-ou-over,Spencer Horwitz Runs Over/Under,SPENCER_HORWITZ_1_MLB,under,0.5,N/A,draftkings,-280,0.5
3,31Qcqt0rYjSZKKkvvSYK,"CINCINNATI_REDS_MLB, PITTSBURGH_PIRATES_MLB",points-SPENCER_HORWITZ_1_MLB-game-ou-under,points-SPENCER_HORWITZ_1_MLB-game-ou-over,Spencer Horwitz Runs Over/Under,SPENCER_HORWITZ_1_MLB,under,0.5,N/A,fliff,-285,0.5
4,31Qcqt0rYjSZKKkvvSYK,"CINCINNATI_REDS_MLB, PITTSBURGH_PIRATES_MLB",points-ELLY_DE_LA_CRUZ_1_MLB-game-ou-over,points-ELLY_DE_LA_CRUZ_1_MLB-game-ou-under,Elly De La Cruz Runs Over/Under,ELLY_DE_LA_CRUZ_1_MLB,over,0.5,N/A,draftkings,+100,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
463,c708SFnAaJRvWD77F9xE,"KANSAS_CITY_ROYALS_MLB, SAN_FRANCISCO_GIANTS_MLB",points-MIKE_MASSEY_1_MLB-game-ou-over,points-MIKE_MASSEY_1_MLB-game-ou-under,Mike Massey Runs Over/Under,MIKE_MASSEY_1_MLB,over,0.5,N/A,fanduel,+165,0.5
464,c708SFnAaJRvWD77F9xE,"KANSAS_CITY_ROYALS_MLB, SAN_FRANCISCO_GIANTS_MLB",points-DUSTIN_RENFROE_1_MLB-game-ou-over,points-DUSTIN_RENFROE_1_MLB-game-ou-under,Hunter Renfroe Runs Over/Under,DUSTIN_RENFROE_1_MLB,over,0.5,N/A,fanduel,+180,0.5
465,c708SFnAaJRvWD77F9xE,"KANSAS_CITY_ROYALS_MLB, SAN_FRANCISCO_GIANTS_MLB",points-CHRISTIAN_KOSS_1_MLB-game-ou-over,points-CHRISTIAN_KOSS_1_MLB-game-ou-under,Christian Koss Runs Over/Under,CHRISTIAN_KOSS_1_MLB,over,0.5,N/A,fanduel,+175,0.5
466,c708SFnAaJRvWD77F9xE,"KANSAS_CITY_ROYALS_MLB, SAN_FRANCISCO_GIANTS_MLB",points-FREDDY_FERMIN_1_MLB-game-ou-over,points-FREDDY_FERMIN_1_MLB-game-ou-under,Freddy Fermin Runs Over/Under,FREDDY_FERMIN_1_MLB,over,0.5,N/A,fanduel,+175,0.5


In [17]:
#getting the last 3 letters of the teams collumn to get MLB
df_odds['sport'] = df_odds['teams'].str[-3:]

In [18]:
# Split into two new columns
df_odds[['team_1', 'team_2']] = df_odds['teams'].str.split(', ', expand=True)
# Remove the "_MLB" suffix from both columns using .loc
df_odds.loc[:, 'team_1'] = df_odds['team_1'].str.replace('_MLB', '', regex=False)
df_odds.loc[:, 'team_2'] = df_odds['team_2'].str.replace('_MLB', '', regex=False)
# Drop the original 'teams' column
df_odds = df_odds.drop(columns=['teams'])

In [19]:
#extract the players name from marketName
df_odds['player'] = df_odds['marketName'].str.extract(r'^(.*?)\s*Runs')

In [20]:
# Define the mapping dictionary
team_mapping = {
    'ARIZONA_DIAMONDBACKS': 'ARI',
    'ATLANTA_BRAVES': 'ATL',
    'BALTIMORE_ORIOLES': 'BAL',
    'BOSTON_RED_SOX': 'BOS',
    'CHICAGO_CUBS': 'CHC',
    'CHICAGO_WHITE_SOX': 'CHW',
    'CINCINNATI_REDS': 'CIN',
    'CLEVELAND_GUARDIANS': 'CLE',
    'COLORADO_ROCKIES': 'COL',
    'DETROIT_TIGERS': 'DET',
    'HOUSTON_ASTROS': 'HOU',
    'KANSAS_CITY_ROYALS': 'KCR',
    'LOS_ANGELES_ANGELS': 'LAA',
    'LOS_ANGELES_DODGERS': 'LAD',
    'MIAMI_MARLINS': 'MIA',
    'MILWAUKEE_BREWERS': 'MIL',
    'MINNESOTA_TWINS': 'MIN',
    'NEW_YORK_METS': 'NYM',
    'NEW_YORK_YANKEES': 'NYY',
    'OAKLAND_ATHLETICS': 'ATH',
    'PHILADELPHIA_PHILLIES': 'PHI',
    'PITTSBURGH_PIRATES': 'PIT',
    'SAN_DIEGO_PADRES': 'SDP',
    'SAN_FRANCISCO_GIANTS': 'SFG',
    'SEATTLE_MARINERS': 'SEA',
    'STLOUIS_CARDINALS': 'STL',
    'TAMPA_BAY_RAYS': 'TBR',
    'TEXAS_RANGERS': 'TEX',
    'TORONTO_BLUE_JAYS': 'TOR',
    'WASHINGTON_NATIONALS': 'WSH'}


# Apply the mapping
df_odds.loc[:, 'team_1'] = df_odds['team_1'].map(team_mapping)
df_odds.loc[:, 'team_2'] = df_odds['team_2'].map(team_mapping)

In [21]:
#making the line collumn an integer
df_odds['overUnder'] = pd.to_numeric(df_odds['overUnder'], errors='coerce')

In [22]:
df_odds['bet']='runs'

In [23]:
df_odds=df_odds[['player','side','overUnder','bookmaker','odds','sport','team_1','team_2','bet']]

In [24]:
# Merge on the 'player' column
df_odds = df_odds.merge(
    batter_team_2025[['player', 'team']],
    on='player',
    how='left'
)


In [25]:
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,team_1,team_2,bet,team
0,Spencer Horwitz,over,0.5,draftkings,+195,MLB,CIN,PIT,runs,PIT
1,Spencer Horwitz,over,0.5,fliff,+190,MLB,CIN,PIT,runs,PIT
2,Spencer Horwitz,under,0.5,draftkings,-280,MLB,CIN,PIT,runs,PIT
3,Spencer Horwitz,under,0.5,fliff,-285,MLB,CIN,PIT,runs,PIT
4,Elly De La Cruz,over,0.5,draftkings,+100,MLB,CIN,PIT,runs,CIN
...,...,...,...,...,...,...,...,...,...,...
462,Sam Huff,over,0.5,fanduel,+165,MLB,KCR,SFG,runs,SFG
464,Hunter Renfroe,over,0.5,fanduel,+180,MLB,KCR,SFG,runs,KCR
465,Christian Koss,over,0.5,fanduel,+175,MLB,KCR,SFG,runs,SFG
466,Freddy Fermin,over,0.5,fanduel,+175,MLB,KCR,SFG,runs,KCR


In [26]:
# Step 1: Identify the "odd one out" team to find the defensive team
df_odds['def_team'] = df_odds.apply(lambda row: row['team_2'] if row['team'] == row['team_1'] else row['team_1'], axis=1)

In [27]:
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,team_1,team_2,bet,team,def_team
0,Spencer Horwitz,over,0.5,draftkings,+195,MLB,CIN,PIT,runs,PIT,CIN
1,Spencer Horwitz,over,0.5,fliff,+190,MLB,CIN,PIT,runs,PIT,CIN
2,Spencer Horwitz,under,0.5,draftkings,-280,MLB,CIN,PIT,runs,PIT,CIN
3,Spencer Horwitz,under,0.5,fliff,-285,MLB,CIN,PIT,runs,PIT,CIN
4,Elly De La Cruz,over,0.5,draftkings,+100,MLB,CIN,PIT,runs,CIN,PIT
...,...,...,...,...,...,...,...,...,...,...,...
462,Sam Huff,over,0.5,fanduel,+165,MLB,KCR,SFG,runs,SFG,KCR
464,Hunter Renfroe,over,0.5,fanduel,+180,MLB,KCR,SFG,runs,KCR,SFG
465,Christian Koss,over,0.5,fanduel,+175,MLB,KCR,SFG,runs,SFG,KCR
466,Freddy Fermin,over,0.5,fanduel,+175,MLB,KCR,SFG,runs,KCR,SFG


In [28]:
#drop redundant collumns that are useless
df_odds = df_odds.drop(columns=['team_1', 'team_2'], errors='ignore')

In [29]:
# Rename the team column before merging (if needed)
def_rank = def_rank.rename(columns={"team": "def_team"})

In [30]:
# Merge on the 'player' column
df_odds = df_odds.merge(def_rank, on='def_team', how='left')
df_odds.dropna()

,player,side,overUnder,bookmaker,odds,sport,bet,team,def_team,Unnamed: 0,def_rank
0,Spencer Horwitz,over,0.5,draftkings,+195,MLB,runs,PIT,CIN,10,11
1,Spencer Horwitz,over,0.5,fliff,+190,MLB,runs,PIT,CIN,10,11
2,Spencer Horwitz,under,0.5,draftkings,-280,MLB,runs,PIT,CIN,10,11
3,Spencer Horwitz,under,0.5,fliff,-285,MLB,runs,PIT,CIN,10,11
4,Elly De La Cruz,over,0.5,draftkings,+100,MLB,runs,CIN,PIT,19,19
...,...,...,...,...,...,...,...,...,...,...,...
462,Sam Huff,over,0.5,fanduel,+165,MLB,runs,SFG,KCR,1,2
464,Hunter Renfroe,over,0.5,fanduel,+180,MLB,runs,KCR,SFG,7,8
465,Christian Koss,over,0.5,fanduel,+175,MLB,runs,SFG,KCR,1,2
466,Freddy Fermin,over,0.5,fanduel,+175,MLB,runs,KCR,SFG,7,8


In [31]:
#making game collumn
df_odds['game'] = df_odds['team'] + ' vs ' + df_odds['def_team']

In [32]:
df_odds2=df_odds.dropna()

In [33]:
df_odds3 = df_odds2.drop(columns=['team', 'def_team'])

In [34]:
overs = df_odds3[(df_odds3['side'].str.lower() == 'over') & (df_odds3['def_rank'] > 15)]
unders = df_odds3[(df_odds3['side'].str.lower() == 'under') & (df_odds3['def_rank'] < 16)]

In [35]:
overs = overs.merge(over_runs_rate, on='player', how='left')
unders = unders.merge(under_runs_rate, on='player', how='left')

In [36]:
overs_2=overs.dropna()
unders_2=unders.dropna()

In [37]:
#making odds collumn numeric
unders_2['odds'] = pd.to_numeric(unders_2['odds'], errors='coerce')
overs_2['odds'] = pd.to_numeric(overs_2['odds'], errors='coerce')

C:\Users\Joshua\AppData\Local\Temp\ipykernel_156340\2110421890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['odds'] = pd.to_numeric(unders_2['odds'], errors='coerce')
C:\Users\Joshua\AppData\Local\Temp\ipykernel_156340\2110421890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['odds'] = pd.to_numeric(overs_2['odds'], errors='coerce')


In [38]:
#making decimal odds
unders_2['decimal_odds'] = unders_2['odds'].apply(
    lambda x: (x / 100) + 1 if x > 0 else (100 / abs(x)) + 1
)
overs_2['decimal_odds'] = overs_2['odds'].apply(
    lambda x: (x / 100) + 1 if x > 0 else (100 / abs(x)) + 1
)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_156340\875580270.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['decimal_odds'] = unders_2['odds'].apply(
C:\Users\Joshua\AppData\Local\Temp\ipykernel_156340\875580270.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['decimal_odds'] = overs_2['odds'].apply(


In [39]:
#making profit
overs_2['profit'] = (overs_2['decimal_odds'] * 100) - 100
unders_2['profit'] = (unders_2['decimal_odds'] * 100) - 100

C:\Users\Joshua\AppData\Local\Temp\ipykernel_156340\3093691362.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['profit'] = (overs_2['decimal_odds'] * 100) - 100
C:\Users\Joshua\AppData\Local\Temp\ipykernel_156340\3093691362.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['profit'] = (unders_2['decimal_odds'] * 100) - 100


In [40]:
def calculate_ev(row):
    if row['overUnder'] == 0.5:
        rate = row['1_runs_rate_over']
    elif row['overUnder'] == 1.5:
        rate = row['2_runs_rate_over']
    elif row['overUnder'] == 2.5:
        rate = row['3_runs_rate_over']
    elif row['overUnder'] == 3.5:
        rate = row['4_runs_rate_over']
    else:
        return None  # or np.nan, if you want to leave it blank for other lines

    # Expected Value formula
    return (rate * row['profit']) - ((1 - rate) * 100)

# Apply the formula to each row
overs_2['edge'] = overs_2.apply(calculate_ev, axis=1)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_156340\1516265758.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overs_2['edge'] = overs_2.apply(calculate_ev, axis=1)


In [41]:
#edge for unders
def calculate_under_ev(row):
    if row['overUnder'] == 0.5:
        rate = row['1_runs_rate_under']
    elif row['overUnder'] == 1.5:
        rate = row['2_runs_rate_under']
    elif row['overUnder'] == 2.5:
        rate = row['3_runs_rate_under']
    elif row['overUnder'] == 3.5:
        rate = row['4_runs_rate_under']
    else:
        return None  # or np.nan for unsupported lines

    return ((1 - rate) * row['profit']) - (rate * 100)

# Apply to each row
unders_2['edge'] = unders_2.apply(calculate_under_ev, axis=1)

C:\Users\Joshua\AppData\Local\Temp\ipykernel_156340\4051107437.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unders_2['edge'] = unders_2.apply(calculate_under_ev, axis=1)


In [42]:
overs_2 = overs_2[overs_2['edge'] > 20]

In [43]:
unders_2 = unders_2[unders_2['edge'] > 20]

In [44]:
#merging overs_2 on top of unders_2
final_df = pd.concat([overs_2, unders_2], ignore_index=True)

In [45]:
final_df=final_df[['player','side','overUnder','bookmaker','odds','sport','bet','edge','game']]

In [46]:
final_df['bet'] = final_df['side'].astype(str) + ' ' + final_df['overUnder'].astype(str) + ' ' + final_df['bet'].astype(str)


In [47]:
final_df = final_df.rename(columns={'bookmaker': 'book'})

In [48]:
final_df['edge'] = final_df['edge'] / 2

In [49]:
final_df['edge'] = final_df['edge'].round(1)

In [50]:
final_df=final_df[['book','sport','player','game','bet','odds','edge']]

In [51]:
final_df = final_df.sort_values(by='edge', ascending=False).reset_index(drop=True)


In [52]:
final_df

,book,sport,player,game,bet,odds,edge
0,underdog,MLB,Christopher Morel,TBR vs HOU,under 0.5 runs,100,33.3
1,prizepicks,MLB,Christopher Morel,TBR vs HOU,under 0.5 runs,100,33.3
2,draftkings,MLB,Spencer Steer,CIN vs PIT,over 0.5 runs,170,26.7
3,fliff,MLB,Christopher Morel,TBR vs HOU,under 0.5 runs,-120,26.4
4,draftkings,MLB,Christopher Morel,TBR vs HOU,under 0.5 runs,-125,25.0
5,fliff,MLB,Spencer Steer,CIN vs PIT,over 0.5 runs,160,23.9
6,prizepicks,MLB,Yandy Diaz,TBR vs HOU,under 0.5 runs,100,20.6
7,draftkings,MLB,Yandy Diaz,TBR vs HOU,under 0.5 runs,-105,19.0
8,fliff,MLB,Yandy Diaz,TBR vs HOU,under 0.5 runs,-105,19.0
9,draftkings,MLB,Danny Jansen,TBR vs HOU,under 0.5 runs,-160,16.7


In [53]:
#putting + in front of the ones that dont have it
final_df['odds'] = final_df['odds'].apply(lambda x: f"+{x}" if x > 0 else str(x))

final_df

,book,sport,player,game,bet,odds,edge
0,underdog,MLB,Christopher Morel,TBR vs HOU,under 0.5 runs,+100,33.3
1,prizepicks,MLB,Christopher Morel,TBR vs HOU,under 0.5 runs,+100,33.3
2,draftkings,MLB,Spencer Steer,CIN vs PIT,over 0.5 runs,+170,26.7
3,fliff,MLB,Christopher Morel,TBR vs HOU,under 0.5 runs,-120,26.4
4,draftkings,MLB,Christopher Morel,TBR vs HOU,under 0.5 runs,-125,25.0
5,fliff,MLB,Spencer Steer,CIN vs PIT,over 0.5 runs,+160,23.9
6,prizepicks,MLB,Yandy Diaz,TBR vs HOU,under 0.5 runs,+100,20.6
7,draftkings,MLB,Yandy Diaz,TBR vs HOU,under 0.5 runs,-105,19.0
8,fliff,MLB,Yandy Diaz,TBR vs HOU,under 0.5 runs,-105,19.0
9,draftkings,MLB,Danny Jansen,TBR vs HOU,under 0.5 runs,-160,16.7


In [54]:
# Convert to JSON list of dicts
data_json = json.loads(final_df.to_json(orient="records"))

# WordPress REST API endpoint
url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_runs"

# Headers
headers = {
    "Content-Type": "application/json"
}

# Send POST request (no auth needed)
response = requests.post(url, headers=headers, json=data_json)

# Print response for debugging
print("Status Code:", response.status_code)
print("Response Text:", response.text)

Status Code: 200
Response Text: {"message":"Data received","data":[{"book":"underdog","sport":"MLB","player":"Christopher Morel","game":"TBR vs HOU","bet":"under 0.5 runs","odds":"+100","edge":33.3},{"book":"prizepicks","sport":"MLB","player":"Christopher Morel","game":"TBR vs HOU","bet":"under 0.5 runs","odds":"+100","edge":33.3},{"book":"draftkings","sport":"MLB","player":"Spencer Steer","game":"CIN vs PIT","bet":"over 0.5 runs","odds":"+170","edge":26.7},{"book":"fliff","sport":"MLB","player":"Christopher Morel","game":"TBR vs HOU","bet":"under 0.5 runs","odds":"-120","edge":26.4},{"book":"draftkings","sport":"MLB","player":"Christopher Morel","game":"TBR vs HOU","bet":"under 0.5 runs","odds":"-125","edge":25},{"book":"fliff","sport":"MLB","player":"Spencer Steer","game":"CIN vs PIT","bet":"over 0.5 runs","odds":"+160","edge":23.9},{"book":"prizepicks","sport":"MLB","player":"Yandy Diaz","game":"TBR vs HOU","bet":"under 0.5 runs","odds":"+100","edge":20.6},{"book":"draftkings","spor

In [55]:
# # Convert DataFrame to JSON (ensure it's a Python list, not a string)
# data_json = json.loads(final_df.to_json(orient="records"))  # ✅ Converts string to Python object

# # WordPress API URL
# url = "https://pitstoppicks.com/wp-json/custom-api/v1/mlb_batter_total_runs"

# # Authentication
# # auth = HTTPBasicAuth("josheskragmailcom", "VKdW RLJu TlWM nE7Z odqH SjYg")
# auth = HTTPBasicAuth("josheskragmailcom", "BFgx z2iv nW9I voXn 1X51 fjRq")
# # Headers
# headers = {
#     "Content-Type": "application/json"
# }

# # Send POST request (requests will handle JSON encoding automatically)
# response = requests.post(url, headers=headers, auth=auth, json=data_json)  # ✅ FIX: Use json= instead of data=

# # Debugging Output
# print("Status Code:", response.status_code)
# print("Response Text:", response.text)

In [56]:
#YOU CAN DELETE ALL OF THIS, THIS IS USED FOR ME RUNNING IT LOCALLY

In [57]:
# Step 1: Sort to ensure reproducibility in tie cases
df_sorted = final_df.sort_values('edge')

# Step 2: Define a function to get the row with median edge for each group
def get_median_row(group):
    median_edge = group['edge'].median()
    # Get the row closest to the median (or first in case of exact match/ties)
    return group.iloc[(group['edge'] - median_edge).abs().argsort().iloc[0]]

# Step 3: Apply it to each group
df_filtered = df_sorted.groupby(['player', 'game', 'bet'], group_keys=False).apply(get_median_row)

# Step 4: Reset index if needed
df_filtered.reset_index(drop=True, inplace=True)
df_filtered

C:\Users\Joshua\AppData\Local\Temp\ipykernel_156340\1266780210.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered = df_sorted.groupby(['player', 'game', 'bet'], group_keys=False).apply(get_median_row)


,book,sport,player,game,bet,odds,edge
0,draftkings,MLB,Andrew McCutchen,PIT vs CIN,under 0.5 runs,-195,10.5
1,fliff,MLB,Christopher Morel,TBR vs HOU,under 0.5 runs,-120,26.4
2,draftkings,MLB,Danny Jansen,TBR vs HOU,under 0.5 runs,-160,16.7
3,draftkings,MLB,Dansby Swanson,CHC vs MIA,over 0.5 runs,+125,10.4
4,fliff,MLB,Spencer Steer,CIN vs PIT,over 0.5 runs,+160,23.9
5,draftkings,MLB,Yandy Diaz,TBR vs HOU,under 0.5 runs,-105,19.0


In [58]:
# Make sure odds column is numeric
df_filtered['odds'] = pd.to_numeric(df_filtered['odds'], errors='coerce')
# Convert American odds to decimal odds
def american_to_decimal(odds):
    if pd.isna(odds):
        return None
    if odds > 0:
        return round((odds / 100) + 1, 3)
    else:
        return round((100 / abs(odds)) + 1, 3)

# Apply conversion
df_filtered['decimal_odds'] = df_filtered['odds'].apply(american_to_decimal)

# Check it out
print(df_filtered[['odds', 'decimal_odds']].head())

   odds  decimal_odds
0  -195         1.513
1  -120         1.833
2  -160         1.625
3   125         2.250
4   160         2.600


In [59]:
df_filtered['profit'] = (df_filtered['decimal_odds'] * 100) - 100
df_filtered

,book,sport,player,game,bet,odds,edge,decimal_odds,profit
0,draftkings,MLB,Andrew McCutchen,PIT vs CIN,under 0.5 runs,-195,10.5,1.513,51.3
1,fliff,MLB,Christopher Morel,TBR vs HOU,under 0.5 runs,-120,26.4,1.833,83.3
2,draftkings,MLB,Danny Jansen,TBR vs HOU,under 0.5 runs,-160,16.7,1.625,62.5
3,draftkings,MLB,Dansby Swanson,CHC vs MIA,over 0.5 runs,125,10.4,2.250,125.0
4,fliff,MLB,Spencer Steer,CIN vs PIT,over 0.5 runs,160,23.9,2.600,160.0
5,draftkings,MLB,Yandy Diaz,TBR vs HOU,under 0.5 runs,-105,19.0,1.952,95.2


In [60]:
df_filtered.to_csv('median_runs_bets.csv')